In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-3.0.0-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
spark = SparkSession\
  .builder\
  .appName("universityranksvsaward")\
  .getOrCreate()

In [ ]:
path = "data/Ranking System II.csv"
ranks = spark.read.csv(path ,header = "True")

In [ ]:
ranks.take(5)

[Row(world_rank='1', university_name='Harvard University', national_rank='1', total_score='100.0', alumni='100.0', award='100.0', hici='100.0', ns='100.0', pub='100.0', pcp='72.4', year='2005'),
 Row(world_rank='2', university_name='University of Cambridge', national_rank='1', total_score='73.6', alumni='99.8', award='93.4', hici='53.3', ns='56.6', pub='70.9', pcp='66.9', year='2005'),
 Row(world_rank='3', university_name='Stanford University', national_rank='2', total_score='73.4', alumni='41.1', award='72.2', hici='88.5', ns='70.9', pub='72.3', pcp='65.0', year='2005'),
 Row(world_rank='4', university_name='University of California, Berkeley', national_rank='3', total_score='72.8', alumni='71.8', award='76.0', hici='69.4', ns='73.9', pub='72.2', pcp='52.7', year='2005'),
 Row(world_rank='5', university_name='Massachusetts Institute of Technology (MIT)', national_rank='4', total_score='70.1', alumni='74.0', award='80.6', hici='66.7', ns='65.8', pub='64.3', pcp='53.0', year='2005')]

In [ ]:
ranks.printSchema()

root
 |-- world_rank: string (nullable = true)
 |-- university_name: string (nullable = true)
 |-- national_rank: string (nullable = true)
 |-- total_score: string (nullable = true)
 |-- alumni: string (nullable = true)
 |-- award: string (nullable = true)
 |-- hici: string (nullable = true)
 |-- ns: string (nullable = true)
 |-- pub: string (nullable = true)
 |-- pcp: string (nullable = true)
 |-- year: string (nullable = true)



In [ ]:
ranks.createOrReplaceTempView("ranks_table")

In [ ]:
spark.sql("select * from ranks_table limit 5").show()

+----------+--------------------+-------------+-----------+------+-----+-----+-----+-----+----+----+
|world_rank|     university_name|national_rank|total_score|alumni|award| hici|   ns|  pub| pcp|year|
+----------+--------------------+-------------+-----------+------+-----+-----+-----+-----+----+----+
|         1|  Harvard University|            1|      100.0| 100.0|100.0|100.0|100.0|100.0|72.4|2005|
|         2|University of Cam...|            1|       73.6|  99.8| 93.4| 53.3| 56.6| 70.9|66.9|2005|
|         3| Stanford University|            2|       73.4|  41.1| 72.2| 88.5| 70.9| 72.3|65.0|2005|
|         4|University of Cal...|            3|       72.8|  71.8| 76.0| 69.4| 73.9| 72.2|52.7|2005|
|         5|Massachusetts Ins...|            4|       70.1|  74.0| 80.6| 66.7| 65.8| 64.3|53.0|2005|
+----------+--------------------+-------------+-----------+------+-----+-----+-----+-----+----+----+



In [ ]:
spark.sql('''
  SELECT * 
  FROM ranks_table
  LIMIT
  5
  '''
   ).show()

+----------+--------------------+-------------+-----------+------+-----+-----+-----+-----+----+----+
|world_rank|     university_name|national_rank|total_score|alumni|award| hici|   ns|  pub| pcp|year|
+----------+--------------------+-------------+-----------+------+-----+-----+-----+-----+----+----+
|         1|  Harvard University|            1|      100.0| 100.0|100.0|100.0|100.0|100.0|72.4|2005|
|         2|University of Cam...|            1|       73.6|  99.8| 93.4| 53.3| 56.6| 70.9|66.9|2005|
|         3| Stanford University|            2|       73.4|  41.1| 72.2| 88.5| 70.9| 72.3|65.0|2005|
|         4|University of Cal...|            3|       72.8|  71.8| 76.0| 69.4| 73.9| 72.2|52.7|2005|
|         5|Massachusetts Ins...|            4|       70.1|  74.0| 80.6| 66.7| 65.8| 64.3|53.0|2005|
+----------+--------------------+-------------+-----------+------+-----+-----+-----+-----+----+----+



In [ ]:
high_ranks = spark.sql('''
   SELECT world_rank,university_name,award,year
   FROM ranks_table
   WHERE year = 2005
''').show()

+----------+--------------------+-----+----+
|world_rank|     university_name|award|year|
+----------+--------------------+-----+----+
|         1|  Harvard University|100.0|2005|
|         2|University of Cam...| 93.4|2005|
|         3| Stanford University| 72.2|2005|
|         4|University of Cal...| 76.0|2005|
|         5|Massachusetts Ins...| 80.6|2005|
|         6|California Instit...| 68.6|2005|
|         7| Columbia University| 60.6|2005|
|         8|Princeton University| 76.8|2005|
|         9|University of Chi...| 81.9|2005|
|        10|University of Oxford| 59.1|2005|
|        11|     Yale University| 44.5|2005|
|        12|  Cornell University| 52.4|2005|
|        13|University of Cal...| 34.7|2005|
|        14|University of Cal...| 32.8|2005|
|        15|University of Pen...| 35.1|2005|
|        16|University of Wis...| 36.3|2005|
|        17|University of Was...| 32.4|2005|
|        18|University of Cal...| 37.6|2005|
|        19|The Johns Hopkins...| 28.3|2005|
|        2

In [ ]:
highest_award = spark.sql('''
   SELECT max(award) as maxaward
   FROM ranks_table
   WHERE year = 2005
''').show()

+--------+
|maxaward|
+--------+
|    93.4|
+--------+



In [ ]:
min_award = spark.sql('''
   SELECT min(award) as minaward
   FROM ranks_table
   WHERE year = 2005
''').show()

+--------+
|minaward|
+--------+
|     0.0|
+--------+



In [ ]:
meanaward = spark.sql('''
   SELECT mean(award) as meanaward
   FROM ranks_table
   WHERE year = 2005 
''').show()

+-----------------+
|        meanaward|
+-----------------+
|6.684800000000008|
+-----------------+



In [ ]:
spark.sql('''
   SELECT world_rank,university_name,award,year
   FROM ranks_table
   WHERE year = 2005
   ORDER BY award asc
''').show()

+----------+--------------------+-----+----+
|world_rank|     university_name|award|year|
+----------+--------------------+-----+----+
|        99|     Lund University|  0.0|2005|
|   101-152|University of Cal...|  0.0|2005|
|   101-152|Baylor College of...|  0.0|2005|
|        93|  University of Utah|  0.0|2005|
|   101-152|Catholic Universi...|  0.0|2005|
|        21|University of Mic...|  0.0|2005|
|   101-152|   Dartmouth College|  0.0|2005|
|        41|University of Cal...|  0.0|2005|
|   101-152|    Emory University|  0.0|2005|
|        47|University of Mar...|  0.0|2005|
|   101-152|Georgia Institute...|  0.0|2005|
|        57|University of Flo...|  0.0|2005|
|   101-152| Hokkaido University|  0.0|2005|
|        32|     Duke University|  0.0|2005|
|   101-152|University of Ams...|  0.0|2005|
|   101-152|   Kyushu University|  0.0|2005|
|        73|   Tohoku University|  0.0|2005|
|   101-152| Mayo Medical School|  0.0|2005|
|        77|Michigan State Un...|  0.0|2005|
|   101-15

In [ ]:
spark.sql('''
   SELECT world_rank,university_name,award,year
   FROM ranks_table
   WHERE year = 2005 and award > 50
   ORDER BY award desc
''').show()

+----------+--------------------+-----+----+
|world_rank|     university_name|award|year|
+----------+--------------------+-----+----+
|         2|University of Cam...| 93.4|2005|
|         9|University of Chi...| 81.9|2005|
|         5|Massachusetts Ins...| 80.6|2005|
|         8|Princeton University| 76.8|2005|
|         4|University of Cal...| 76.0|2005|
|         3| Stanford University| 72.2|2005|
|         6|California Instit...| 68.6|2005|
|         7| Columbia University| 60.6|2005|
|        30|Rockefeller Unive...| 59.8|2005|
|        10|University of Oxford| 59.1|2005|
|        12|  Cornell University| 52.4|2005|
|         1|  Harvard University|100.0|2005|
+----------+--------------------+-----+----+



In [ ]:
column_names = ['world_rank','university_name','national_rank','total_score','alumni','award','hici','ns','pub','pcp','year']

In [ ]:
ranks[(ranks.world_rank < 6) & (ranks.award > 50)]

DataFrame[world_rank: string, university_name: string, national_rank: string, total_score: string, alumni: string, award: string, hici: string, ns: string, pub: string, pcp: string, year: string]

In [ ]:
ranks.groupby('world_rank')['award'].mean().plot()

TypeError: ignored